In [88]:
import tensorflow as tf
# Make sure the GPU is enabled
assert tf.config.list_physical_devices('GPU'), 'Start the colab kernel with GPU: Runtime -> Change runtime type -> GPU'

In [89]:
!pip install pyfaidx
!pip install pybedtools
!pip install kipoiseq==0.5.2 --quiet > /dev/null

In [90]:
import pandas as pd
import pyfaidx
import pybedtools
import tensorflow_hub as hub
import kipoiseq
from kipoiseq import Interval
from kipoiseq.transforms.functional import resize_interval
import numpy as np

# Setup

In [91]:
transform_path = 'gs://dm-enformer/models/enformer.finetuned.SAD.robustscaler-PCA500-robustscaler.transform.pkl'
model_path = 'https://tfhub.dev/deepmind/enformer/1'
fasta_file = '/root/data/genome.fa'
EPCrisprBenchmark_path = 'https://raw.githubusercontent.com/owenyi2/Garvan_Internship_2023/main/using_ENFORMER/EPCrisprBenchmark_ensemble_data_reference.tsv'

In [92]:
targets_txt = 'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_human.txt'
df_targets = pd.read_csv(targets_txt, sep='\t')
df_targets.iloc[[4828, 5111], :]

,index,genome,identifier,file,clip,scale,sum_stat,description
4828,4828,0,CNhs11250,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:chronic myelogenous leukemia cell line:K562
5111,5111,0,CNhs12336,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:chronic myelogenous leukemia cell line:K5...


In [93]:
!mkdir -p /root/data
!wget -O - http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz | gunzip -c > {fasta_file}
pyfaidx.Faidx(fasta_file)
!ls /root/data

--2024-01-12 04:45:09--  http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 983659424 (938M) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 938.09M  16.1MB/s    in 58s     

2024-01-12 04:46:08 (16.1 MB/s) - written to stdout [983659424/983659424]

genome.fa  genome.fa.fai


## Setup CRISPR_df

In [94]:
CRISPRi_df = pd.read_csv(EPCrisprBenchmark_path, sep="\t")
assert((CRISPRi_df["chrom"] == CRISPRi_df["chrTSS"]).all()) # Assert all Enhancer Promoter pairs lie on same chromosome
assert((CRISPRi_df["endTSS"] - CRISPRi_df["startTSS"] == 1).all()) # Assert that all Transcription Start Sites are defined at 1bp long

CRISPRi_df["startTSS"] = CRISPRi_df["startTSS"].astype(int)
CRISPRi_df["endTSS"] = CRISPRi_df["endTSS"].astype(int)

CRISPRi_df["orientation"] = (CRISPRi_df["chromStart"] + CRISPRi_df["chromEnd"]) // 2 < CRISPRi_df["startTSS"]
CRISPRi_df["distance"] = np.maximum(np.abs(CRISPRi_df["chromStart"] - CRISPRi_df["startTSS"]), np.abs(CRISPRi_df["chromEnd"] - CRISPRi_df["startTSS"]))
CRISPRi_df["distanceClass"] = np.where(CRISPRi_df["distance"] < 57344, 0, np.where(CRISPRi_df["distance"] < 114688, 1, 2))
CRISPRi_df

,dataset,chrom,chromStart,chromEnd,name,EffectSize,chrTSS,startTSS,endTSS,measuredGeneSymbol,Significant,pValueAdjusted,PowerAtEffectSize25,ValidConnection,CellType,Reference,Regulated,orientation,distance,distanceClass
0,FlowFISH_K562,chr1,3774714,3775214,CEP104|chr1:3691278-3691778:*,-0.293432,chr1,3857213,3857214,CEP104,True,0.004024,0.825094,True,K562,Ulirsch2016,True,True,82499,1
1,FlowFISH_K562,chr1,3774714,3775214,LRRC47|chr1:3691278-3691778:*,-0.331178,chr1,3796503,3796504,LRRC47,True,0.007771,0.608994,True,K562,Ulirsch2016,True,True,21789,0
2,FlowFISH_K562,chr1,3774714,3775214,SMIM1|chr1:3691278-3691778:*,-0.472019,chr1,3772761,3772762,SMIM1,True,0.000642,0.639332,True,K562,Ulirsch2016,True,False,2453,0
3,FlowFISH_K562,chr10,125816658,125816735,UROS|chr10:127505227-127505304:*,-0.850000,chr10,125823284,125823285,UROS,True,NaN,0.234322,True,K562,Wakabayashi2016,True,True,6626,0
4,FlowFISH_K562,chr11,3261795,3262295,HBE1|chr11:3283025-3283525:*,0.029242,chr11,5505651,5505652,HBE1,False,0.944569,1.000000,True,K562,Klann2017,False,True,2243856,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10406,TAPseq,chr8,125528738,125529164,NSMCE2|chr8:125528738-125529164:.,0.018247,chr8,125091679,125091680,NSMCE2,False,1.000000,1.000000,True,K562,"Schraivogel et al., 2020",False,False,437485,2
10407,TAPseq,chr8,125544507,125545059,NSMCE2|chr8:125544507-125545059:.,0.085396,chr8,125091679,125091680,NSMCE2,False,1.000000,1.000000,True,K562,"Schraivogel et al., 2020",False,False,453380,2
10408,TAPseq,chr8,125664274,125664878,NSMCE2|chr8:125664274-125664878:.,-0.021983,chr8,125091679,125091680,NSMCE2,False,1.000000,1.000000,True,K562,"Schraivogel et al., 2020",False,False,573199,2
10409,TAPseq,chr8,125694248,125694539,NSMCE2|chr8:125694248-125694539:.,0.020683,chr8,125091679,125091680,NSMCE2,False,1.000000,1.000000,True,K562,"Schraivogel et al., 2020",False,False,602860,2


In [95]:
# need to classify into within 57344 (0), within 114688 (1), else (2)
CRISPRi_df["distanceClass"] = np.where(CRISPRi_df["distance"] < 57344, 0, np.where(CRISPRi_df["distance"] < 114688, 1, 2))
CRISPRi_df_0 = CRISPRi_df[CRISPRi_df["distanceClass"] == 0].copy()
CRISPRi_df_1 = CRISPRi_df[CRISPRi_df["distanceClass"] == 1].copy()

CRISPRi_df_0.shape, CRISPRi_df_1.shape

((1095, 20), (947, 20))

In [96]:
CRISPRi_df_0["binStart"] = ((CRISPRi_df_0["chromStart"] - CRISPRi_df_0["startTSS"]) // 128 + 448) # map Enhancer position relative to Promoter to a bin
CRISPRi_df_0["binEnd"] = ((CRISPRi_df_0["chromEnd"] - CRISPRi_df_0["startTSS"]) // 128 + 448 + 1)

assert((CRISPRi_df_0["binEnd"] <= 896).all())
assert((CRISPRi_df_0["binStart"] >=0).all())
CRISPRi_df_0

,dataset,chrom,chromStart,chromEnd,name,EffectSize,chrTSS,startTSS,endTSS,measuredGeneSymbol,...,PowerAtEffectSize25,ValidConnection,CellType,Reference,Regulated,orientation,distance,distanceClass,binStart,binEnd
1,FlowFISH_K562,chr1,3774714,3775214,LRRC47|chr1:3691278-3691778:*,-0.331178,chr1,3796503,3796504,LRRC47,...,0.608994,True,K562,Ulirsch2016,True,True,21789,0,277,282
2,FlowFISH_K562,chr1,3774714,3775214,SMIM1|chr1:3691278-3691778:*,-0.472019,chr1,3772761,3772762,SMIM1,...,0.639332,True,K562,Ulirsch2016,True,False,2453,0,463,468
3,FlowFISH_K562,chr10,125816658,125816735,UROS|chr10:127505227-127505304:*,-0.850000,chr10,125823284,125823285,UROS,...,0.234322,True,K562,Wakabayashi2016,True,True,6626,0,396,397
81,FlowFISH_K562,chr11,5476135,5476675,HBE1|chr11:5497365-5497905:*,0.000650,chr11,5505651,5505652,HBE1,...,0.999997,True,K562,Klann2017,False,True,29516,0,217,222
82,FlowFISH_K562,chr11,5506995,5507615,HBE1|chr11:5528225-5528845:*,0.035755,chr11,5505651,5505652,HBE1,...,0.979048,True,K562,Klann2017,False,False,1964,0,458,464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,TAPseq,chr8,124527176,124527667,NDUFB9|chr8:124527176-124527667:.,0.017630,chr8,124539101,124539102,NDUFB9,...,1.000000,True,K562,"Schraivogel et al., 2020",False,True,11925,0,354,359
10288,TAPseq,chr8,124530505,124530795,NDUFB9|chr8:124530505-124530795:.,-0.045380,chr8,124539101,124539102,NDUFB9,...,0.950000,True,K562,"Schraivogel et al., 2020",False,True,8596,0,380,384
10295,TAPseq,chr8,124583789,124584022,NDUFB9|chr8:124583789-124584022:.,-0.004246,chr8,124539101,124539102,NDUFB9,...,1.000000,True,K562,"Schraivogel et al., 2020",False,False,44921,0,797,799
10299,TAPseq,chr8,124584149,124584714,NDUFB9|chr8:124584149-124584714:.,-0.073005,chr8,124539101,124539102,NDUFB9,...,1.000000,True,K562,"Schraivogel et al., 2020",False,False,45613,0,799,805


In [97]:
CRISPRi_df_1["binStart"] = (CRISPRi_df_1["chromStart"] - CRISPRi_df_1["startTSS"]) // 128 + np.where(CRISPRi_df_1["orientation"], 896, 0)
CRISPRi_df_1["binEnd"] = (CRISPRi_df_1["chromEnd"] - CRISPRi_df_1["startTSS"]) // 128 + np.where(CRISPRi_df_1["orientation"], 896, 0) + 1

assert((CRISPRi_df_1["binEnd"] <= 896).all())
assert((CRISPRi_df_1["binStart"] >=0).all())
CRISPRi_df_1

,dataset,chrom,chromStart,chromEnd,name,EffectSize,chrTSS,startTSS,endTSS,measuredGeneSymbol,...,PowerAtEffectSize25,ValidConnection,CellType,Reference,Regulated,orientation,distance,distanceClass,binStart,binEnd
0,FlowFISH_K562,chr1,3774714,3775214,CEP104|chr1:3691278-3691778:*,-0.293432,chr1,3857213,3857214,CEP104,...,0.825094,True,K562,Ulirsch2016,True,True,82499,1,251,256
84,FlowFISH_K562,chr11,5591615,5592135,HBE1|chr11:5612845-5613365:*,0.060036,chr11,5505651,5505652,HBE1,...,1.000000,True,K562,Klann2017,False,False,86484,1,671,676
85,FlowFISH_K562,chr11,5600255,5600775,HBE1|chr11:5621485-5622005:*,-0.018648,chr11,5505651,5505652,HBE1,...,1.000000,True,K562,Klann2017,False,False,95124,1,739,744
251,FlowFISH_K562,chr12,54168961,54169521,HNRNPA1|chr12:54562745-54563305:*,-0.066725,chr12,54280193,54280194,HNRNPA1,...,0.999993,True,K562,Fulco2019,False,True,111232,1,27,32
255,FlowFISH_K562,chr12,54175341,54175841,HNRNPA1|chr12:54569125-54569625:*,-0.034361,chr12,54280193,54280194,HNRNPA1,...,0.999665,True,K562,Fulco2019,False,True,104852,1,76,81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10319,TAPseq,chr8,124645615,124645826,NDUFB9|chr8:124645615-124645826:.,-0.051871,chr8,124539101,124539102,NDUFB9,...,1.000000,True,K562,"Schraivogel et al., 2020",False,False,106725,1,832,834
10320,TAPseq,chr8,124649001,124649543,NDUFB9|chr8:124649001-124649543:.,0.015918,chr8,124539101,124539102,NDUFB9,...,1.000000,True,K562,"Schraivogel et al., 2020",False,False,110442,1,858,863
10321,TAPseq,chr8,124649599,124649959,NDUFB9|chr8:124649599-124649959:.,0.022421,chr8,124539101,124539102,NDUFB9,...,1.000000,True,K562,"Schraivogel et al., 2020",False,False,110858,1,863,867
10322,TAPseq,chr8,124651086,124651884,NDUFB9|chr8:124651086-124651884:.,-0.045480,chr8,124539101,124539102,NDUFB9,...,0.950000,True,K562,"Schraivogel et al., 2020",False,False,112783,1,874,882


# Model


In [98]:
SEQUENCE_LENGTH = 393216

class Enformer:

    def __init__(self, tfhub_url):
        self._model = hub.load(tfhub_url).model

    def predict_on_batch(self, inputs):
        predictions = self._model.predict_on_batch(inputs)
        return {k: v.numpy() for k, v in predictions.items()}

    @tf.function
    def contribution_input_grad(self, input_sequence,
                                target_mask, output_head='human'):

        input_sequence = input_sequence[tf.newaxis]


        target_mask_mass = tf.reduce_sum(target_mask)
        with tf.GradientTape() as tape:
            tape.watch(input_sequence)
            prediction = tf.reduce_sum(
                target_mask[tf.newaxis] *
                self._model.predict_on_batch(input_sequence)[output_head]) / target_mask_mass

        grad = tape.gradient(prediction, input_sequence)

        input_grad = grad * input_sequence
        input_grad = tf.squeeze(input_grad, axis=0)

        return tf.reduce_sum(input_grad, axis=-1)

    @tf.function
    def contribution_input_grad_batch(self, input_sequence,
                                target_mask, output_head='human'):

        target_mask_mass = tf.reduce_sum(target_mask)
        with tf.GradientTape() as tape:
            tape.watch(input_sequence)
            prediction = tf.reduce_sum(
                target_mask *
                self._model.predict_on_batch(input_sequence)[output_head]) / target_mask_mass

        grad = tape.gradient(prediction, input_sequence)

        input_grad = grad * input_sequence

        return tf.reduce_sum(input_grad, axis=-1)




In [99]:
class FastaStringExtractor:

    def __init__(self, fasta_file):
        self.fasta = pyfaidx.Fasta(fasta_file)
        self._chromosome_sizes = {k: len(v) for k, v in self.fasta.items()}

    def extract(self, interval: Interval, **kwargs) -> str:
        # Truncate interval if it extends beyond the chromosome lengths.
        chromosome_length = self._chromosome_sizes[interval.chrom]
        trimmed_interval = Interval(interval.chrom,
                                    max(interval.start, 0),
                                    min(interval.end, chromosome_length),
                                    )
        # pyfaidx wants a 1-based interval
        sequence = str(self.fasta.get_seq(trimmed_interval.chrom,
                                          trimmed_interval.start + 1,
                                          trimmed_interval.stop).seq).upper()

        # Fill truncated values with N's.
        pad_upstream = 'N' * max(-interval.start, 0)
        pad_downstream = 'N' * max(interval.end - chromosome_length, 0)
        return pad_upstream + sequence + pad_downstream

    def close(self):
        return self.fasta.close()

def one_hot_encode(sequence):
    return kipoiseq.transforms.functional.one_hot_dna(sequence).astype(np.float32)


In [100]:
model = Enformer(model_path)
fasta_extractor = FastaStringExtractor(fasta_file)

# Contribution Scores

In [101]:
def resize(interval, width, anchor="centre"):
    if anchor == "start":
        return kipoiseq.Interval(interval.chrom, interval.start, interval.start + width)
    if anchor == "end":
        return kipoiseq.Interval(interval.chrom, interval.end - width, interval.end)
    if anchor == "centre":
        centre = (interval.start + interval.end) // 2

        return kipoiseq.Interval(interval.chrom, centre - width // 2 - width % 2, centre + width // 2)

In [102]:
# Checking things work (ignore this cell)

target_interval = kipoiseq.Interval(CRISPRi_df_0["chrTSS"].iloc[0], CRISPRi_df_0["startTSS"].iloc[0], CRISPRi_df_0["endTSS"].iloc[0])
sequence_one_hot = one_hot_encode(fasta_extractor.extract(resize(target_interval, SEQUENCE_LENGTH, "centre")))

target_mask = np.zeros((896, len(df_targets))).astype(np.float32)
for idx in [447, 448, 449]:
    target_mask[idx, 4828] = 1
    target_mask[idx, 5111] = 1

contribution_scores = model.contribution_input_grad(sequence_one_hot.astype(np.float32), target_mask).numpy()
pooled_contribution_scores = tf.nn.avg_pool1d(np.abs(contribution_scores)[np.newaxis, :, np.newaxis], 128, 128, 'VALID')[0, :, 0].numpy()[1088:-1088]
np.mean(pooled_contribution_scores[CRISPRi_df_0["binStart"].iloc[0]:CRISPRi_df_0["binEnd"].iloc[0]])

0.22445479

In [106]:
def calculate_contribution_score(row, df): # could try taking advantage of batch predict, but we don't have enough GPU ram for anything more than like 5 at a time
    if df["distanceClass"].iloc[row] == 0:
        mask = [447, 448, 449]
        anchor = "centre"
    elif df["distanceClass"].iloc[row] == 1:
        if df["orientation"].iloc[row]: # I.e. Enhancer comes before Promoter (Enhancer < Promoter)
            mask = [893, 894, 895]
            anchor = "end"
        else:
            mask = [0, 1, 2]
            anchor = "start"
    assert(df["distanceClass"].iloc[row] in [0, 1]) # yeah um don't shove in if distanceClass == 2 i.e. enhancer too far from promoter

    target_interval = kipoiseq.Interval(df["chrTSS"].iloc[row], df["startTSS"].iloc[row], df["endTSS"].iloc[row])
    sequence_one_hot = one_hot_encode(fasta_extractor.extract(resize(target_interval, SEQUENCE_LENGTH, anchor)))

    target_mask = np.zeros((896, len(df_targets))).astype(np.float32)
    for idx in mask:
        target_mask[idx, 4828] = 1
        target_mask[idx, 5111] = 1

    contribution_scores = model.contribution_input_grad(sequence_one_hot.astype(np.float32), target_mask).numpy()
    pooled_contribution_scores = tf.nn.avg_pool1d(np.abs(contribution_scores)[np.newaxis, :, np.newaxis], 128, 128, 'VALID')[0, :, 0].numpy()[1088:-1088]

    return np.mean(pooled_contribution_scores[df["binStart"].iloc[row]:df["binEnd"].iloc[row]])



## CRISPRi_df_0
Candidate pairs whose distance between Enh/Pro is within 57344 (i.e. 896*128 // 2)

896 being the # of output bins and 128 being the width (bp) of each bin

In [86]:
output_list = []

for i in range(len(CRISPRi_df_0)):
    result = calculate_contribution_score(i, CRISPRi_df_0)
    print(CRISPRi_df_0.index[i], CRISPRi_df_0["name"].iloc[i], result, CRISPRi_df_0["Regulated"].iloc[i])
    output_list.append(result)

CRISPRi_df_0["ENFORMER_score"] = output_list

1 LRRC47|chr1:3691278-3691778:* 0.22445479 True
2 SMIM1|chr1:3691278-3691778:* 1.6962675 True
3 UROS|chr10:127505227-127505304:* 2.1088321 True
81 HBE1|chr11:5497365-5497905:* 0.10583589 False
82 HBE1|chr11:5528225-5528845:* 0.27718323 False
83 HBE1|chr11:5552225-5553025:* 0.5433938 False
131 FADS1|chr11:61602067-61602390:* 0.011659292 True
132 FADS1|chr11:61609256-61609274:* 0.0029492378 True
299 HNRNPA1|chr12:54621105-54621625:* 0.03760745 False
303 HNRNPA1|chr12:54621725-54622225:* 0.056593496 False
308 COPZ1|chr12:54645365-54645945:* 0.037720002 False
309 HNRNPA1|chr12:54645365-54645945:* 0.11944055 False
311 NFE2|chr12:54645365-54645945:* 0.04420111 False
312 COPZ1|chr12:54655565-54656105:* 0.031106288 False
313 HNRNPA1|chr12:54655565-54656105:* 0.07189737 False
315 NFE2|chr12:54655565-54656105:* 0.028611919 False
316 COPZ1|chr12:54656205-54656725:* 0.052043162 False
317 HNRNPA1|chr12:54656205-54656725:* 0.08763947 False
319 NFE2|chr12:54656205-54656725:* 0.056425624 False
320 COP

In [117]:
CRISPRi_df_0.to_csv("ENFORMER_output_0.tsv", sep="\t")
CRISPRi_df_0

,dataset,chrom,chromStart,chromEnd,name,EffectSize,chrTSS,startTSS,endTSS,measuredGeneSymbol,...,ValidConnection,CellType,Reference,Regulated,orientation,distance,distanceClass,binStart,binEnd,ENFORMER_score
1,FlowFISH_K562,chr1,3774714,3775214,LRRC47|chr1:3691278-3691778:*,-0.331178,chr1,3796503,3796504,LRRC47,...,True,K562,Ulirsch2016,True,True,21789,0,277,282,0.224455
2,FlowFISH_K562,chr1,3774714,3775214,SMIM1|chr1:3691278-3691778:*,-0.472019,chr1,3772761,3772762,SMIM1,...,True,K562,Ulirsch2016,True,False,2453,0,463,468,1.696268
3,FlowFISH_K562,chr10,125816658,125816735,UROS|chr10:127505227-127505304:*,-0.850000,chr10,125823284,125823285,UROS,...,True,K562,Wakabayashi2016,True,True,6626,0,396,397,2.108832
81,FlowFISH_K562,chr11,5476135,5476675,HBE1|chr11:5497365-5497905:*,0.000650,chr11,5505651,5505652,HBE1,...,True,K562,Klann2017,False,True,29516,0,217,222,0.105836
82,FlowFISH_K562,chr11,5506995,5507615,HBE1|chr11:5528225-5528845:*,0.035755,chr11,5505651,5505652,HBE1,...,True,K562,Klann2017,False,False,1964,0,458,464,0.277183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,TAPseq,chr8,124527176,124527667,NDUFB9|chr8:124527176-124527667:.,0.017630,chr8,124539101,124539102,NDUFB9,...,True,K562,"Schraivogel et al., 2020",False,True,11925,0,354,359,0.240818
10288,TAPseq,chr8,124530505,124530795,NDUFB9|chr8:124530505-124530795:.,-0.045380,chr8,124539101,124539102,NDUFB9,...,True,K562,"Schraivogel et al., 2020",False,True,8596,0,380,384,0.052964
10295,TAPseq,chr8,124583789,124584022,NDUFB9|chr8:124583789-124584022:.,-0.004246,chr8,124539101,124539102,NDUFB9,...,True,K562,"Schraivogel et al., 2020",False,False,44921,0,797,799,0.047139
10299,TAPseq,chr8,124584149,124584714,NDUFB9|chr8:124584149-124584714:.,-0.073005,chr8,124539101,124539102,NDUFB9,...,True,K562,"Schraivogel et al., 2020",False,False,45613,0,799,805,0.038022


## CRISPRi_df_1
Candidate pairs whose distance between Enh/Pro is within 114688 (i.e. 896*128) but not within 57344

896 being the # of output bins and 128 being the width (bp) of each bin

In [107]:
calculate_contribution_score(0, CRISPRi_df_1)

1.8608129e-05

In [ ]:
# Untested so far
output_list = []

for i in range(len(CRISPRi_df_1)):
    result = calculate_contribution_score(i, CRISPRi_df_1)
    print(CRISPRi_df_1.index[i], CRISPRi_df_1["name"].iloc[i], result, CRISPRi_df_1["Regulated"].iloc[i])
    output_list.append(result)

CRISPRi_df_1["ENFORMER_score"] = output_list